# Harmony OPeNDAP SubSetter (HOSS)

### Contact:

* **Slack:** #variable_subsetter
* **Jira:** [SDPS Data Services](https://bugs.earthdata.nasa.gov/secure/RapidBoard.jspa?rapidView=757&view=planning.nodetail)

## What is HOSS?

HOSS is the [Harmony OPeNDAP SubSetter](https://github.com/nasa/harmony-opendap-subsetter); a Harmony backend service that translates a request specified by the user into an [OPeNDAP-compatible constraint expression](https://opendap.github.io/documentation/UserGuideComprehensive.html#Constraint_Expressions), to retrieve only the data requested by the user. HOSS is written in Python, and exists within a Docker image, hosted in [the GitHub container registry](https://github.com/nasa/harmony-opendap-subsetter/pkgs/container/harmony-opendap-subsetter).

HOSS heavily relies on a combination of CF-Convention metadata, and the [earthdata-varinfo package](https://github.com/nasa/earthdata-varinfo) to retrieve the variables a user requests, as well as additional variables contained in the CF-Convention references and variable dimension scales of those requested variables. This ensures that the end-user receives a viable output product, where the returned pixels retain spatial and temporal location information.

## Capabilities

* Variable subset (what you need, in addition to what you ask for)
* Bounding box spatial subset.
* Spatial subsetting for a GeoJSON shape file.
* Temporal subset (single range).
* Named-dimension subsetting.
* Preservation of input granule hierarchy.
* NetCDF-4 output.

## Requirements

* L3 or L4 data, with CF-Convention compliant metadata for grid dimensions.
* Data accessible via Hyrax.
* Hierarchical granules have `.dmrpp` files with `<Group>` elements.
* UMM-Var records associated with the collection.
* An OPeNDAP UMM-S record, associated with the UMM-C record(s) for your collection(s).

## How HOSS processes a request

<img src="figures/HOSS_sequence_diagram.png" width="600px" />

For all requests:

* HOSS receives a Harmony message, which can specify requested variables and/or a bounding box spatial extent. HOSS also receives a STAC catalog entry containing the OPeNDAP URL for the granule.
* HOSS extracts the relevant information from the message and STAC catalog entry.
* HOSS makes a request to Hyrax for the Dataset Metadata Response (DMR) file associated with the granule.
* HOSS parses the DMR file using `earthdata-varinfo`, creating a mapping of variable dependencies.

For variable subset requests:

* HOSS extracts all required variables to support those requested using the `earthdata-varinfo` mapping. For example, spatially gridded variables will require their grid dimension variables, such as latitude, longitude and time.

For spatial, temporal or named-dimension subset requests:

* HOSS performs a prefetch of grid dimension variables, in full, from Hyrax.
* HOSS translates the dimension extents as defined in the Harmony request into dimension array indices. For example, a bounding box with -60 ≤ latitude (degrees north) ≤ -40 will be found to correspond to the $M^{\rm{th}}$ and $N^{\rm{th}}$ index in the `latitude` dimension array.
* HOSS appends array index ranges to all gridded variables to be retrieved from Hyrax. For example: `latitude[M:N]`, `longitude[X:Y]` and `science_variable[][M:N][X:Y]`. In this example, `science_variable` has grid dimensions `time`, `latitude` and `longitude`, in that order. However, HOSS does not rely on variable dimensions to be in a specific order.

Then, for all requests:

* HOSS creates a DAP4 constraint expression for the final variable and/or bounding box spatial subset.
* HOSS makes a request for the final NetCDF-4 output. (Note - if a bounding box is requested, and it crosses the grid edge, filling will be applied before the results are staged)
* HOSS stages the results using standard Harmony functionality.

When a GeoJSON shape file is included in a request, or a bounding box is specified for projection-gridded data, HOSS is combined with the MaskFill Harmony backend service to create a polygon spatial subset. In both cases, HOSS identifies the spatial dimension indices that define a minimally encompassing bounding box for the shape, and requests this from OPeNDAP. MaskFill is then used to fill those pixels that are within this bounding box, but outside of the polygon. Note, a geographically defined bounding box will not be rectangular for some non-geographic projections.

## Examples

The following examples will use [harmony-py](https://github.com/nasa/harmony-py) to make requests to Harmony. `harmony-py` is distributed via the Python Package Index ([PyPI](https://pypi.org/project/harmony-py/)), and can be installed to your current Python environment via Pip:

```bash
$ pip install harmony-py
```

### Initial set-up

The following cell imports standard classes from `harmony-py` and establishes a client to make requests to the UAT environment of Harmony. It also sets up two collection instances:

* RSSMIF16D, an ocean product offered by GHRC: [C1238392622-EEDTEST](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1238392622-EEDTEST.html).
* GPM3_IMERGHH, a precipitation product offered by GES-DISC: [C1245618475-EEDTEST](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1245618475-EEDTEST.html).

In this notebook, clones of the collections hosted in the EEDTEST CMR provider will be used.

In [ ]:
from harmony import BBox, Client, Collection, Environment, Request


harmony_client = Client(env=Environment.UAT)

demo_directory = '.'
ghrc_collection = Collection(id='C1238392622-EEDTEST')
ghrc_granule_id = 'G1245840464-EEDTEST'
gpm_collection = Collection(id='C1245618475-EEDTEST')
gpm_granule_id = 'G1245649588-EEDTEST'

### A variable subset:

This first request will retrieve data from the RSSMIF16D GHRC ocean product. Asking for the `atmosphere_cloud_liquid_water_content` should retrieve this variable, along with the associated dimension variables:

* `latitude`
* `longitude`
* `time`

In [ ]:
variables = ['atmosphere_cloud_liquid_water_content']

variable_subset_request = Request(collection=ghrc_collection, variables=variables, granule_id=[ghrc_granule_id])
variable_subset_job_id = harmony_client.submit(variable_subset_request)

print(f'Processing job: {variable_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(variable_subset_job_id, overwrite=True, directory=demo_directory)]:
    print(f'Downloaded: {filename}')

### A bounding box subset:

This request will perform a bounding box spatial subset for the GPM IMERG half hourly collection. No variable subset will be applied, so all 16 variables should be retrieved.

In [ ]:
gpm_bounding_box = BBox(w=45, s=-45, e=75, n=-15)

bbox_request = Request(collection=gpm_collection, spatial=gpm_bounding_box, granule_id=[gpm_granule_id])
bbox_job_id = harmony_client.submit(bbox_request)

print(f'Processing job: {bbox_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(bbox_job_id, overwrite=True, directory=demo_directory)]:
    print(f'Downloaded: {filename}')

### Combined variable and bounding box subset:

Using the same bounding box from before, the next request will specify only the `/Grid/precipitationCal` variable. This should retrieve 7 variables:

* `/Grid/precipitationCal` - the requested variable.
* `/Grid/lat` - a grid dimension variable.
* `/Grid/lon` - a grid dimension variable.
* `/Grid/time` - grid dimension variable.
* `/Grid/lat_bnds` - a CF-Convention `bounds` reference, denoting pixel start and end value.
* `/Grid/lon_bnds` - a CF-Convention `bounds` reference, denoting pixel start and end value.
* `/Grid/time_bnds` - a CF-Convention `bounds` reference, denoting pixel start and end value.

In [ ]:
gpm_bounding_box = BBox(w=45, s=-45, e=75, n=-15)
gpm_variables = ['/Grid/precipitationCal']

combined_request = Request(collection=gpm_collection, spatial=gpm_bounding_box,
                           granule_id=[gpm_granule_id], variables=gpm_variables)
combined_job_id = harmony_client.submit(combined_request)

print(f'Processing job: {combined_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(combined_job_id, overwrite=True, directory=demo_directory)]:
    print(f'Downloaded: {filename}')

### A bounding box that crosses a grid edge:

It is possible for a user to specify a bounding box that crosses the longitudinal edge of a geographic grid. For example, the RSSMIF16D collection has its grid edge at the Prime Meridian. In these cases, HOSS will retrieve the requested latitude range of data, but the full range of longitude data in that latitude range. HOSS will then fill the longitude ranges outside the requested bounding box. The diagram below illustrates the data that are retrieved:

<img src="figures/HOSS_crossing_edge_annotated.png" width="400px" />

In [ ]:
ghrc_bounding_box = BBox(w=-30, s=-50, e=30, n=0)

edge_request = Request(collection=ghrc_collection, spatial=ghrc_bounding_box, granule_id=[ghrc_granule_id])
edge_job_id = harmony_client.submit(edge_request)

print(f'Processing job: {edge_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(edge_job_id, overwrite=True, directory=demo_directory)]:
    print(f'Downloaded: {filename}')

### A single point request:

It is possible to specify a bounding box where the western and eastern extents are equal, as are the northern and southern extents. The returned data will depend on where exactly the point is with respect to the pixel grid:

* If the point lies within a pixel, only that pixel will be returned.
* If the point lies on the corner where four pixels meet, all four surrounding pixels will be returned.
* If the point lies on the edge of two pixels (midway along that boundary), the two touching pixels will be returned.

These requests require `harmony-py > 0.3.0`

#### Example inside a pixel:

In [ ]:
point_in_pixel_box = BBox(w=43.2222, s=-25.1111, e=43.2222, n=-25.1111)
gpm_variables = ['/Grid/precipitationCal']

point_in_pixel_request = Request(collection=gpm_collection, spatial=point_in_pixel_box,
                                 granule_id=[gpm_granule_id], variables=gpm_variables)
point_in_pixel_job_id = harmony_client.submit(point_in_pixel_request)

print(f'Processing job: {point_in_pixel_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(point_in_pixel_job_id, overwrite=True, directory=demo_directory)]:
    print(f'Downloaded: {filename}')

#### Example at pixel corner:

In [ ]:
corner_point_box = BBox(w=160, s=20, e=160, n=20)
gpm_variables = ['/Grid/precipitationCal']

corner_point_request = Request(collection=gpm_collection, spatial=corner_point_box,
                                 granule_id=[gpm_granule_id], variables=gpm_variables)
corner_point_job_id = harmony_client.submit(corner_point_request)

print(f'Processing job: {corner_point_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(corner_point_job_id, overwrite=True, directory=demo_directory)]:
    print(f'Downloaded: {filename}')